# Creating manual annotations and training Cellpose
*General tip 1*: Whenever you modify anything in this notebook, do not forget to click on the save button. In this way, all changes will be saved and you will be able to continue working on your project later on.

*General tip 2*: To run a part of this notebook, you should first click on a section/cell (after selecting it, you should see a blue bar on the left side) and then click the Play button above (alternatively, press *Shift+Enter*). After running a cell, the next one will be automatically selected.

## 1) Select project directory
Here you have two options:

- If you are *starting a new project*, select an empty folder: all generated data and labels will be saved in this folder.
- If you want to *keep working with an exhisting project*, then select the previous project folder: pre-existing data will be automatically loaded.

To get the path of the selected project directory, you can click on the "Browse..." button below. Then you will be able to copy the displayed path in the code-cell below (see comments below).

In [2]:
%load_ext autoreload
%autoreload 2

import os
import ipywidgets as widgets
from ipywidgets import Button
from tkinter import Tk, filedialog
from IPython.display import clear_output, display

selected_directory = widgets.Label(value="")


def select_proj_dir(b):
    clear_output()
    root = Tk()
    root.withdraw()  # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)  # Raise the root to the top of all windows.
    b.directory = filedialog.askdirectory()
    # b.files = filedialog.askopenfilename(multiple=True) # List of selected files will be set button's file attribute.
    selected_directory.value = "Copy this path below: \"{}\"".format(b.directory)
    # with out_proj_dir:        
    #     print("Selected project directory: {}".format(b.directory))


fileselect = Button(description="Browse...")
fileselect.on_click(select_proj_dir)

widgets.VBox(
    [widgets.HBox([widgets.Label(value="Click to select a project directory: "), fileselect]), selected_directory])

In [3]:
# !!! Do not forget to re-run this cell and the following one every time you change any path !!!

# --------------------  TO BE FILLED BY USER - START  -----------------------------

# Insert path to project directory here:
project_directory = "/Users/alberto-mac/EMBL_ATeam/cellpose_training_pipeline/test_project"  # For example: "/home/my_home/data/my_first_project"

# ---------------------  TO BE FILLED BY USER - END  ------------------------------

In [4]:
# Validate project directory:
assert os.path.isdir(project_directory), "The selected proj dir '{}' is not a directory!"

# TODO: load data from directory and list crops

Explain: current crops are listed above

## 2) Select regions of interest to be annotated
**Important**: You only need to run section (2) of the notebook if you want to *add* or *modify* regions of interest. If you have already selected all the region of interests you need, then you can move on to section 3.

In this section, you will load an image and then select one or more region of interests (rectangular boxes) that you will manually annotate in the following and will be used to train the segmentation algorithm. 

### Things to keep in mind when selecting region of interest
- When you select a region of interest, keep in mind that **you will need to manually annotate all cells inside that region of interest**. Each region of interest should contain **at least several dozens of cells, ideally ~100**.
- These region of interests will be used to train the segmentation algorithm, so you should select the most representative parts of your data. If there are some particularly challenging parts of your data the you would like the segmentation algorithm to properly segment, then you should also select some regions of interest from these difficult parts. 

Here you have some example scenarios:
1. Your images/datasets present several artifacts close to the borders, but you only care to have an accurate segmentation in the central part of the image, where image quality is higher and there are no artifacts. In this case, you can select regions of interest only from the central area of your image.
2. Your images/datasets present several artifacts in all parts of the image, and you would like cells to be properly segmented even in parts of the image where these artifacts are present. Then, you should also include regions of interest where some of the artifacts are visible (on top of region of interests that are easier to segment)
3. You acquired several images that look somehow different (different acquisition method, brightness, etc). Then, you should evenly select regions of interest across all your images/datasets.

### 2.1) Load the image

#### Option 1: Add/modify crops from a previously loaded image
From the Drop-list menu below, select the image you want to work with. Then, go to step 2.2.

#### Option 2: Add crops for a new image

In [4]:
selected_file = widgets.Label(value="")


def select_file(b):
    clear_output()
    root = Tk()
    root.withdraw()  # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)  # Raise the root to the top of all windows.
    b.file = filedialog.askopenfilename()  # List of selected files will be set button's file attribute.
    selected_file.value = "Copy selected path below: \"{}\" ".format(b.file)


fileselect = Button(description="Browse files...")
fileselect.on_click(select_file)

widgets.VBox([widgets.HBox([widgets.Label(value="To get the path of an image, you can click on this button: "), fileselect]), selected_file])

Please fill the code section below with the paths of the following images:

- Main image that should be segmented: this is usually the bright-field or GFP channel
- Optional: image with DAPI channel (if you have it, then add it, because it can be very useful for the segmentation algorithm)

If you have additional channels that may helpful for doing a precise manual annotation, you can load two additional channels... TODO break code in several blocks...

In [5]:
# !!! Do not forget to re-run this cell and the following one every time you change any path !!!

# --------------------  TO BE FILLED BY USER - START  -----------------------------

# Main image to be segmented:
main_image_path = "/Users/alberto-mac/EMBL_ATeam/cellpose_training_pipeline/test_images/img1/fused_tp_0_ch_4.tif"

use_dapi_channel_for_segmentation = False

# For additional (optional) channels, you can either specify the image paths...
dapi_image_path = ""
extra_ch_1_image_path = ""
extra_ch_2_image_path = ""

# ...or you can define some name filters, so that additional channels are automatically found in the same
# directory of the main image:
filter_main_image = "_ch_4"
filter_dapi_image = "_ch_2"
filter_extra_ch_1 = "_ch_1"
filter_extra_ch_2 = ""

# If you want, you can also define the extra channel names, for clarity:
extra_ch_1_name = "GFP"
extra_ch_2_name = "Extra channel 2"

# ---------------------  TO BE FILLED BY USER - END  ------------------------------

In [6]:
# Validate main image:
assert os.path.isfile(main_image_path), "'{}' is not a file!"


def validate_ch_paths(ch_name, ch_path, name_filter):
    ch_path = None if ch_path == "" else ch_path
    name_filter = None if name_filter == "" else name_filter
    if ch_path is not None:
        assert os.path.isfile(ch_path), "'{}' is not a file!"
    else:
        if name_filter is not None:
            assert isinstance(filter_main_image, str) and filter_main_image != "", "Please insert a proper filter string for main image"
            assert isinstance(name_filter, str) and name_filter != "", "Please insert a proper filter string for the channel {}".format(ch_name)
            ch_path = main_image_path.replace(filter_main_image, name_filter)
            assert os.path.isfile(ch_path), "'{}' is not a file!"
    return ch_path


# Validate DAPI image:
dapi_image_path = validate_ch_paths("DAPI", dapi_image_path, filter_dapi_image)
if use_dapi_channel_for_segmentation:
    assert dapi_image_path is not None

# Validate extra channels:
extra_ch_1_image_path = validate_ch_paths(extra_ch_1_name, extra_ch_1_image_path, filter_extra_ch_1)
extra_ch_2_image_path = validate_ch_paths(extra_ch_2_name, extra_ch_2_image_path, filter_extra_ch_2)


### 2.2) Modify/create region of interest using Napari
After you run the next cell, the selected image will be loaded in Napari. 

Then select the "Rectangle" tool in napari to draw one or more region of interests.

After you are done, close the Napari window and run the  the next sections 

- Select, move, and delete boxes
- Insert image of Napari viewer? (Showing the various buttons)

![Napari](./napari-screenshot.jpg)

In [5]:
from segmfriends.io.images import read_uint8_img
import napari

# FIXME: remove
main_image_path = "/Users/alberto-mac/EMBL_ATeam/cellpose_training_pipeline/test_images/img1/fused_tp_0_ch_4.tif"

# Load images into memory:
main_image = read_uint8_img(main_image_path)

# create the viewer and display the image
viewer = napari.view_image(main_image, name="Main image")

# Load images in napari:
s_layer = viewer.add_shapes(name="Crops", shape_type="rectangle", opacity=0.15,  edge_color='#fff01dff', face_color='#f9ffbeff')
# draw some rectangles, then inspect
s_layer.data

uint8 6 173


[]

In [6]:
s_layer.data

[array([[ 461.92600267,  663.39119173],
        [ 461.92600267, 2062.02415858],
        [1160.03468561, 2062.02415858],
        [1160.03468561,  663.39119173]]),
 array([[ 116.49506267, 1820.46406068],
        [ 116.49506267, 2603.11877788],
        [ 664.83648491, 2603.11877788],
        [ 664.83648491, 1820.46406068]])]

In [7]:
# create the viewer and display the image

# Load images in napari:
s_layer_2 = viewer.add_shapes(s_layer.data, name="Crops 2", shape_type="rectangle", opacity=0.15,  edge_color='#fff01dff', face_color='#f9ffbeff')



In [11]:


global a1
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')


def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a * b * c))


out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

In [ ]:
img_main = widgets.Text(
    value='',
    placeholder='Enter directory path',
    disabled=False
)

widgets.HBox([widgets.VBox([a, b, c]), out])

In [8]:
print(a.value)

42


In [12]:

caption = widgets.Label(value='The selected .')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')


def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )


slider.observe(handle_slider_change, names='value')

display(caption, slider)

Label(value='The selected .')

IntSlider(value=1, description='Slider', max=5, min=-5)

In [13]:
int_range = widgets.IntSlider()
output2 = widgets.Output()

display(int_range, output2)


def on_value_change(change):
    with output2:
        print(change['new'])


int_range.observe(on_value_change, names='value')

IntSlider(value=0)

Output()

In [14]:
import os

proj_dir = fileselect.directory
assert os.path.isdir(proj_dir), "You did not select one single project directory!"
print("Selected project directory: {}".format(proj_dir))

Selected project directory: /Users/alberto-mac/EMBL_ATeam/cellpose_training_pipeline/test_project


In [1]:
#@markdown ###Path to training images:

Training_source = ""  #@param {type:"string"}
Training_target = ""  #@param {type:"string"}

In [2]:
from tkinter import *
from tkinter import ttk
from tkinter import filedialog


def main():
    self = Tk()

    F1 = LabelFrame(self, text="Select File")
    F1.grid(row=0, column=0, padx=3)

    browse = Button(F1, text="Browse...", command=openfile)
    browse.grid(row=0, column=2, padx=1, pady=3)

    E1 = Entry(F1, text="")
    E1.grid(row=0, column=1, sticky="ew")

    L1 = Label(F1, text="Filename:")
    L1.grid(row=0, column=0, padx=3)

    B1 = Button(F1, text="Use This File", command=go)
    B1.grid(row=1, column=2, padx=3, pady=3)

    B2 = Button(F1, text="Cancel", width=7)
    B2.grid(row=1, column=1, sticky="e")

    self.mainloop()


def openfile():
    global filename
    filename = filedialog.askopenfilename()
    E1.delete(0, END)
    E1.insert(0, filename)


def go():
    global filename
    file = open(filename)
    file_content = file.read()
    print(file_content)
    file.close()


main()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/alberto-mac/miniconda3/envs/nifty/lib/python3.8/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/xw/tcy65hq13818_4913hhp666h0000gq/T/ipykernel_14966/967351048.py", line 31, in openfile
    E1.delete(0, END)
NameError: name 'E1' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/alberto-mac/miniconda3/envs/nifty/lib/python3.8/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/xw/tcy65hq13818_4913hhp666h0000gq/T/ipykernel_14966/967351048.py", line 31, in openfile
    E1.delete(0, END)
NameError: name 'E1' is not defined
